## Horn model

1. Create a series of vectors, same rotation from previous plane each time.
2. Draw a circle on the reference plane with decreasing radius each step.

In [1]:
import numpy as np
import k3d
from phyloshape.shape.src.core import Vertex, Vector

### GENERATE DATA

In [2]:
def generate_horn_path(
    length: float=5, 
    curve_radius_start: float=1,
    curve_radius_end: float=None,
    curve_x: float=1, 
    curve_y: float=1, 
    rotation: float=2 * np.pi,
    vector: np.ndarray=None,
    num_intervals: int=20,
) -> np.ndarray:
    """Generate points along a spiral in x, y, z forming the multi-vector path that
    the beak will follow.

    Parameters
    ----------
    length: float
        length of vector beak will follow (rotate around)
    curve_radius_start: float
        radius of circle around which beak is rotating at start
    curve_radius_end: float
        radius of circle around which beak is rotating at end
    curve_x: float
        multiplier of cosine rotation (0 - 1)
    curve_y: float
        multiplier of sin rotation (0 - 1)
    rotation: ndarray
        rotation of beak around circle in radians
    vector: ndarray
        unit vector path of beak; default=[0, 0, 1]
    num_intervals: int
        number of intervals at which points are sampled along vector
    """
    # defaults
    curve_radius_end = curve_radius_start if curve_radius_end is None else curve_radius_end
    vector = np.array([0, 0, 1]) if vector is None else vector

    # curve_[x,y] scale the cos vs sin function between -1 and 1 magnitudes
    curve_x = min(1, max(-1, curve_x))
    curve_y = min(1, max(-1, curve_y))
    
    # generate points for the surface
    theta = np.linspace(0, rotation, num_intervals)
    curve_radii = np.linspace(curve_radius_start, curve_radius_end, num_intervals)
    x = curve_x * curve_radii * np.cos(theta)
    y = curve_y * curve_radii * np.sin(theta)
    z = np.linspace(0, length, num_intervals)

    # Create the point cloud and center first point on [0, 0, 0]
    point_cloud = np.column_stack((x, y, z)) 
    point_cloud -= point_cloud[0]

    # todo: rotate by vector ...
    # ...
    return point_cloud 


In [3]:
def get_distance_ratio(p0, p1, p2, p3):
    """get ratio of rx and ry of circle"""
    verts = [Vertex(i, j) for i, j in enumerate([p0, p1, p2, p3])]
    v0 = Vector(verts[0], verts[2])
    v1 = Vector(verts[1], verts[3])
    return v0.dist / v1.dist

In [4]:
def generate_points_on_circle(radius, num_points, vector, origin, correct=False):
    """Generate N points along the radius of a circle oriented in 3D space
    by a vector, with the specified origin position.
    
    Parameters
    -----------
    radius (float):
        Radius of the circle.
    num_points (int):
        Number of points to generate.
    vector (ndarray): 
        3D vector representing the orientation of the circle.
    origin (ndarray):
        3D vector representing the origin position of the circle.
    """
    theta_test = np.array([0, np.pi / 2, np.pi, np.pi * 3/2])

    # get rotation matrix
    theta = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    u = np.cross(vector, [1, 0, 0])
    if np.linalg.norm(u) < 1e-6:
        u = np.cross(vector, [0, 1, 0])
    u = u / np.linalg.norm(u)
    v = np.cross(vector, u)

    # ensure it is a circle instead of ellipse
    if correct:
        points = (
            origin + radius * np.outer(np.cos(theta_test), u)
            + radius * np.outer(np.sin(theta_test), v)
        )
        ratio = get_distance_ratio(*points)
        v *= ratio

    # get points along circumference of circle
    points = (
        origin 
        + radius * np.outer(np.cos(theta), u)
        + radius * np.outer(np.sin(theta), v)
    )
    return points


### PLOT FUNCTIO

In [5]:
def draw_beak(
    length: float = 5,
    curve_radius_start: float = 1,
    curve_radius_end: float = None,
    curve_x: float = 1,
    curve_y: float = 1,
    rotation: float = 2 * np.pi,
    beak_radius_start: float = 1,
    beak_radius_end: float = 0.1,
    num_intervals: int=20,
):
    """Return a k3d plot of ...

    """
    p = k3d.plot()#camera_mode="orbit")

    # get vertices along beak path
    horn_path = generate_horn_path(
        length=length, 
        curve_radius_start=curve_radius_start,
        curve_radius_end=curve_radius_end,
        curve_x=curve_x,
        curve_y=curve_y,
        rotation=rotation,
        num_intervals=num_intervals,
    )

    # get radius of beak at each interval along path
    radii = np.geomspace(beak_radius_start, beak_radius_end, horn_path.shape[0])

    # get Vertex object at each point on beak path
    vertices = [Vertex(i, horn_path[i]) for i in range(horn_path.shape[0])]

    # ...
    for i in range(len(vertices) - 1):
        v = Vector(vertices[i], vertices[i + 1])

        #skew = 4 if not (xcurve or ycurve) else min(4, radius) #3#4
        cpoints = generate_points_on_circle(
            radius=radii[i],
            num_points=20,
            vector=v.absolute,
            origin=vertices[i].coords,
            correct=True,
        ).astype(np.float32)

        # qrt = int(cpoints.shape[0] / 4)
        # p += k3d.points(cpoints[-qrt:], point_size=0.1, opacity=0.5, color=0x008B8B)
        # p += k3d.points(cpoints[:qrt], point_size=0.1, opacity=0.5, color=0x008B8B)
        # p += k3d.points(cpoints[qrt:-qrt], point_size=0.1, opacity=0.5, color=0x8B008B)

        #p += k3d.vectors(vertices[i].coords.astype(np.float32), v.absolute.astype(np.float32))
        p += k3d.points(cpoints, point_size=0.1, color=0x000000, opacity=0.35)
        p += k3d.points(cpoints[0], point_size=0.2, color=0x008B8B)
        p += k3d.points(cpoints[int(cpoints.shape[0] / 2)], point_size=0.2, color=0x8B008B)

    return p

### TEST

In [7]:
# groenlandica
p = draw_beak(length=8, curve_radius_start=0.5, rotation=np.pi * 2.5, num_intervals=50)

In [ ]:
# dichotoma, anas
draw_beak(length=3, rotation=np.pi * 3/2, curve_x=0, beak_radius_end=0.05, num_intervals=50)

In [ ]:
# fetisowii 
draw_beak(rotation=np.pi * 2.5, curve_radius_start=1, curve_radius_end=0.5, beak_radius_start=1, beak_radius_end=0.1, num_intervals=50)

In [ ]:
# cranolopha
draw_beak(length=5, rotation=np.pi * 3/2, curve_radius_start=1.5, curve_radius_end=1, beak_radius_start=1.5, beak_radius_end=0.2, num_intervals=50)

In [ ]:
# densispica
draw_beak(length=1, rotation=np.pi / 2, curve_x=0, curve_y=1, beak_radius_end=0.2)

In [ ]:
# integrifolia
draw_beak(length=7, rotation=np.pi * 3.25, curve_x=0, curve_y=1, num_intervals=50, beak_radius_start=1.25)

### Generate beaks

In [45]:
from loguru import logger

In [58]:
import phyloshape
phyloshape.set_log_level("DEBUG")
logger = logger.bind(name="phyloshape")

🐞 logger_setup:set_log_level | phyloshape v.0.0.1 logging enabled


In [59]:
logger.info("DHI")

ℹ️ 3646038683:<module>        | DHI


In [60]:
logger.debug("HI")

🐞 1025496370:<module>        | HI


In [ ]:
import pandas as pd
import itertools
data = list(itertools.product(
    [1, 4, 6],             # length
    [np.pi, 2 * np.pi, 0], # rotation
    [-1, 0, 1],            # curve_x
    [-1, 0, 1],            # curve_y
    [2, 4],                # curve_radius_start
    [1, 2],                # beak_radius_start
))
data = pd.DataFrame(data, columns=["length", "rotation", "curve_x", "curve_y", "curve_radius_start", "beak_radius_start"])
data.head(10)

In [34]:
from pathlib import Path

p = Path("~/bitmap.png").expanduser().resolve()

In [44]:
2023-05-17 15:47:39.912 | INFO     | __main__:parse_runinfo:205 - F {'cluster': 'public', 'filename': 'mpg_L5733-1_BO15-052_ACTTGA_L005_R1_001.fastq.gz', 'size': '4609807739', 'date': '2020-03-05 16:24:05', 'md5': 'dc57c012a771195815c402fb6fe00702', 'version': '1', 'semantic_name': 'fastq', 'supertype': 'Original', 'sratoolkit': '0'}
2023-05-17 15:47:39.913 | INFO     | __main__:parse_runinfo:205 - F {'cluster': 'public', 'filename': 'mpg_L5733-1_BO15-052_ACTTGA_L005_R2_001.fastq.gz', 'size': '4812098287', 'date': '2020-03-05 16:31:47', 'md5': '117386db08026f9e5012a8be2942763a', 'version': '1', 'semantic_name': 'fastq', 'supertype': 'Original', 'sratoolkit': '0'}
2023-05-17 15:47:39.913 | INFO     | __main__:parse_runinfo:205 - F {'cluster': 'public', 'filename': 'SRR6670194', 'url': 'https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR6670194/SRR6670194', 'size': '5352621398', 'date': '2018-02-05 06:56:20', 'md5': '0d5ae614990e692d040c2e18cf01a0e4', 'version': '1', 'semantic_name': 'SRA Normalized', 'supertype': 'Primary ETL', 'sratoolkit': '1'}
2023-05-17 15:47:39.914 | INFO     | __main__:parse_runinfo:205 - F {'cluster': 'public', 'filename': 'SRR6670194.sralite', 'url': 'https://sra-downloadb.be-md.ncbi.nlm.nih.gov/sos5/sra-pub-zq-11/SRR006/670/SRR6670194/SRR6670194.sralite.1', 'size': '4112376558', 'date': '2020-06-23 21:46:06', 'md5': 'c181c72f9abedb7a37c5cddc59ecaadd', 'version': '1', 'semantic_name': 'SRA Lite', 'supertype': 'Primary ETL', 'sratoolkit': '1'}

PosixPath('/home/deren/bitmap.png/hi/bo')

In [20]:
Path("/tmp/davidii.html").expanduser().resolve()

PosixPath('/tmp/davidii.html')

In [ ]:
# davidii
draw_beak(length=5, rotation=-np.pi * 3/2, curve_radius_start=1.5, curve_radius_end=1, beak_radius_start=1.5, beak_radius_end=0.2, num_intervals=50)

### Multivariate simulation

In [ ]:
from scipy import stats

In [ ]:
data.columns

In [ ]:
corr = np.array([
    [1.0,   0.7,  0.5,  0.5, -0.5, -0.5],
    [0.7,   1.0,  0.5,  0.5,  0.0, -0.1],
    [0.5,   0.5,  1.0,  0.2,  0.0, -0.2],
    [0.5,   0.5,  0.2,  1.0,  0.0, -0.2],
    [-0.5,  0.0,  0.0,  0.0,  1.0,  0.5],
    [-0.5, -0.1, -0.2, -0.2,  0.5,  1.0],
])

In [ ]:
# convert corr matrix to cov matrix
std_deviations = np.sqrt(np.diag(corr))
std_matrix = np.diag(std_deviations)
covariance_matrix = std_matrix @ corr @ std_matrix
covariance_matrix

In [ ]:
mean = np.array([5, 0, 0, 0, 2, 1.5])
rvs = stats.multivariate_normal.rvs(mean, covariance_matrix)
rvs

In [ ]:
# random
rvs = stats.multivariate_normal.rvs(mean, covariance_matrix)
kwargs = dict(zip(["length", "rotation", "curve_x", "curve_y", "beak_radius_start"], rvs))
print(kwargs)
draw_beak(num_intervals=50, **kwargs)

In [ ]:
np.pi